In [1]:
# Installs the necessary Python and system libraries
try:
    from easypip import easyimport, easyinstall, is_notebook
except ModuleNotFoundError as e:
    get_ipython().run_line_magic("pip", "install easypip")
    from easypip import easyimport, easyinstall, is_notebook

easyinstall("bbrl>=0.2.2")
easyinstall("swig")
easyinstall("bbrl_gymnasium>=0.2.0")
easyinstall("bbrl_gymnasium[box2d]")
easyinstall("bbrl_gymnasium[classic_control]")
easyinstall("tensorboard")
easyinstall("moviepy")
easyinstall("box2d-kengz")

In [2]:
import os
import sys
from pathlib import Path
import math

from moviepy.editor import ipython_display as video_display
import time
from tqdm.auto import tqdm
from typing import Tuple, Optional
from functools import partial

from omegaconf import OmegaConf
import torch
import bbrl_gymnasium

import copy
from abc import abstractmethod, ABC
import torch.nn as nn
import torch.nn.functional as F
from time import strftime
OmegaConf.register_new_resolver(
    "current_time", lambda: strftime("%Y%m%d-%H%M%S"), replace=True
)
from bbrl.agents.gymnasium import GymAgent, ParallelGymAgent, make_env, record_video

In [3]:
from datetime import datetime
from datetime import date

import base64
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

import gymnasium
from gymnasium import logger as gymlogger
# from gym.wrappers import Monitor # deprecated 2023 - https://stackoverflow.com/questions/71520568/importerror-cannot-import-name-monitor-from-gym-wrappers
from gymnasium.wrappers.record_video import RecordVideo
gymlogger.set_level(40) #error only
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

print("\n",date.today(), datetime.now().strftime("%H:%M:%S"),"GMT") # timestamp is greenwich time
print("OK.")

def show_video(loop=True, num=0):
    mp4list = glob.glob(f'videoTest/rl-video-episode-{num}.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        if loop == True:
            ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:videoTest/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
        else:
            ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    controls style="height: 400px;">
                    <source src="data:videoTest/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
    
def wrap_env(env):
    env = RecordVideo(env, './videoTest',  episode_trigger = lambda episode_number: True) # !!! 2023
    env.reset() # !!! 2023
    #env = Monitor(env, './video', force=True)
    return env


 2024-01-10 15:36:36 GMT
OK.


In [4]:
"""
Continuous action version of the classic cart-pole system implemented by Rich
Sutton et al.
"""
import math

import numpy as np

from gymnasium.envs.classic_control.cartpole import CartPoleEnv
from gymnasium import logger, spaces
from gymnasium.wrappers import TimeLimit

import sys
sys.modules[__name__]

class ContinuousEnvCACLA(CartPoleEnv):
    """Continuous version  of the CartPole-v1 environment"""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.min_action = -1.0
        self.max_action = 1.0
        self.action_space = spaces.Box(
            self.min_action, self.max_action, shape=(1,), dtype=np.float64
        )

    def step(self, action):
        if action > self.max_action:
            action = np.array(self.max_action)
        elif action < self.min_action:
            action = np.array(self.min_action)
        assert self.state is not None, "Call reset before using step method."

        x, x_dot, theta, theta_dot = self.state
        force = self.force_mag * float(action)

        costheta = math.cos(theta)
        sintheta = math.sin(theta)

        # For the interested reader:
        # https://coneural.org/florian/papers/05_cart_pole.pdf
        temp = (
            force + self.polemass_length * theta_dot**2 * sintheta
        ) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / (
            self.length * (4.0 / 3.0 - self.masspole * costheta**2 / self.total_mass)
        )
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass

        if self.kinematics_integrator == "euler":
            x = x + self.tau * x_dot
            x_dot = x_dot + self.tau * xacc
            theta = theta + self.tau * theta_dot
            theta_dot = theta_dot + self.tau * thetaacc
        else:  # semi-implicit euler
            x_dot = x_dot + self.tau * xacc
            x = x + self.tau * x_dot
            theta_dot = theta_dot + self.tau * thetaacc
            theta = theta + self.tau * theta_dot

        self.state = (x, x_dot, theta, theta_dot)

        terminated = bool(
            x < -self.x_threshold
            or x > self.x_threshold
            or theta < -self.theta_threshold_radians
            or theta > self.theta_threshold_radians
        )

        if not terminated:
            reward = 1.0
        elif self.steps_beyond_terminated is None:
            # Pole just fell!
            self.steps_beyond_terminated = 0
            reward = -1.0
        else:
            if self.steps_beyond_terminated == 0:
                logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned terminated = True. You "
                    "should always call 'reset()' once you receive 'terminated = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_terminated += 1
            reward = 0

        if self.render_mode == "human":
            self.render()
        return np.array(self.state, dtype=np.float32), reward, terminated, False, {}

try:
  gymnasium.envs.register(
      id='CartpoleEnvCacla',
      entry_point='__main__:ContinuousEnvCACLA',
      max_episode_steps=500
  )
except:
    print("Except")
    pass



In [10]:
import math

import numpy as np

from gymnasium.envs.classic_control.cartpole import CartPoleEnv
from gymnasium import logger, spaces

class ContinuousCartPoleEnv(CartPoleEnv):
    """Continuous version  of the CartPole-v1 environment"""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.min_action = -1.0
        self.max_action = 1.0
        self.action_space = spaces.Box(
            self.min_action, self.max_action, shape=(1,), dtype=np.float64
        )

    def step(self, action):
        if action > self.max_action:
            action = np.array(self.max_action)
        elif action < self.min_action:
            action = np.array(self.min_action)
        assert self.state is not None, "Call reset before using step method."

        x, x_dot, theta, theta_dot = self.state
        force = self.force_mag * float(action)

        costheta = math.cos(theta)
        sintheta = math.sin(theta)

        # For the interested reader:
        # https://coneural.org/florian/papers/05_cart_pole.pdf
        temp = (
            force + self.polemass_length * theta_dot**2 * sintheta
        ) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / (
            self.length * (4.0 / 3.0 - self.masspole * costheta**2 / self.total_mass)
        )
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass

        if self.kinematics_integrator == "euler":
            x = x + self.tau * x_dot
            x_dot = x_dot + self.tau * xacc
            theta = theta + self.tau * theta_dot
            theta_dot = theta_dot + self.tau * thetaacc
        else:  # semi-implicit euler
            x_dot = x_dot + self.tau * xacc
            x = x + self.tau * x_dot
            theta_dot = theta_dot + self.tau * thetaacc
            theta = theta + self.tau * theta_dot

        self.state = (x, x_dot, theta, theta_dot)

        terminated = bool(
            x < -self.x_threshold
            or x > self.x_threshold
            or theta < -self.theta_threshold_radians
            or theta > self.theta_threshold_radians
        )

        if not terminated:
            reward = 1.0
        elif self.steps_beyond_terminated is None:
            # Pole just fell!
            self.steps_beyond_terminated = 0
            reward = -1.0
        else:
            if self.steps_beyond_terminated == 0:
                logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned terminated = True. You "
                    "should always call 'reset()' once you receive 'terminated = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_terminated += 1
            reward = 0.0

        if self.render_mode == "human":
            self.render()
        return np.array(self.state, dtype=np.float32), reward, terminated, False, {}

In [33]:
import torch.nn
from torch.nn.functional import mse_loss
import numpy as np
class Cacla(nn.Module):
    def __init__(self, in_dim, h_dim, activation=nn.Tanh, discount_factor=0.95, gaussian_noise=0.01):
        super(Cacla,self).__init__()
        self.critic = nn.Sequential(
            nn.Linear(in_dim, h_dim),
            activation(),
            nn.Linear(h_dim, 1)
        )
        
        self.actor = nn.Sequential(
            nn.Linear(in_dim, h_dim),
            activation(),
            nn.Linear(h_dim, 1),
            activation()
        )
        
        self.discount_factor = discount_factor
        self.gaussian_noise = gaussian_noise
    
    def forward(self, x):
        x = torch.from_numpy(x).unsqueeze(0)
        return self.critic(x), self.actor(x)
    
    def sample(self, x):
        return (x + np.random.normal(0,self.gaussian_noise))

def test(eval_env, model, n_test = 10):
    cum_reward = 0
    with torch.no_grad():
        for _ in range(n_test):
            done = False
            truncated = False
            obs0,_ = eval_env.reset()
            while not done and not truncated:
                _, a0 = model(obs0)  
                obs0, reward, done,truncated,_ = eval_env.step(a0) 
                cum_reward += reward
    return cum_reward/n_test

def addnoise(x, std):
    return (x + np.random.normal(0,std))

def train(eval_env, train_env, model, optim, eval_interval=1000, step_max=100000, noise_std=0.3):
    it = 0
    obs,_ = train_env.reset()
    while it < step_max:
        it += 1
        optim.zero_grad()
        v0, a0 = model(obs)  
        action = model.sample(a0).detach()
        obs, reward, done, truncated,reste = train_env.step(addnoise(action, noise_std))
        v1, _ = model(obs)
        delta = addnoise(reward, noise_std) + model.discount_factor*v1.detach() - v0
        (delta**2).backward()
        if delta > 0: ((action - a0)**2).backward()
        optim.step()
        if done or truncated: obs,_ = train_env.reset()
        if it % eval_interval == 0:
            perf = test(eval_env, model)
            print(f'{it = } | reward {perf}')

In [37]:
from torch.optim import SGD, Adam
# Initialisation
train_env = gymnasium.make('CartpoleEnvCacla')
train_env = TimeLimit(train_env, max_episode_steps=500)
eval_env = gymnasium.make('CartpoleEnvCacla')
eval_env = TimeLimit(eval_env, max_episode_steps=500)

cacla = Cacla(in_dim=4, h_dim=12, 
              discount_factor=0.95, 
              gaussian_noise=0.1)
optim = SGD(cacla.parameters(), lr=0.01)
train(eval_env, train_env, cacla, optim, eval_interval=5000, step_max=102400, noise_std=0.3)

it = 5000 | reward 46.0
it = 10000 | reward 57.5
it = 15000 | reward 61.0
it = 20000 | reward 45.4
it = 25000 | reward 93.5
it = 30000 | reward 99.2


KeyboardInterrupt: 

In [272]:
def addnoise(x, std):
    return x + np.random.normal(0,std)
addnoise(0, 0.1)

-0.04216796513431444